[![imagenes](imagenes/pythonista.png)](https://pythonista.io)

# Algunas funciones útiles de *flask*.

El paquete flask incluye varios componentes que facilitan el desarrollo y control de servicios web, tanto del lado del cliente como del servidor.

Existen módulos especializados en el envío de recursos tales como archivos y flujos de datos, así como de presentación de contenido tradicional mediante páginas en HTML.

En este capítulo se explorará la funcionalidad de algunos de estos objetos.

## La función *abort()*.

Esta función detiene una sesión enviando el código de estado de HTTP ingresado como primer argumento. También es posible añadir un mensaje como segundo argumento.

``` python
abort(<número de mensaje de estado>, <objeto str>)
```

**Ejemplo:**
 La función *usuario()* desplegará un mensaje en HTML si la cadena de caracteres ingresada en ```http://localhost:5000/<cadena de caracteres>``` es exactamente un elemento del objeto tipo *list* llamado *nombres*. En caso contrario emitirá un mensaje de estado *404*.
 * La URL http://localhost:5000/Luis desplegará la página.
 * La URL http://localhost:5000/luis desplegará un error 404.
 
 **Advertencia:** Una vez ejecutada la siguiente celda, es necesario interrumpir el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
from flask import abort, Flask
app = Flask(__name__)
nombres = ['Hugo', 'Paco', 'Luis']

    
@app.route('/<nombre>')
def usuario(nombre):
    if nombre in nombres:
        return '<h1>Hola, {}.</h1>'.format(nombre)
    else:
        abort(404, 'No encontrado.')

#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run('0.0.0.0')

## Definición de URL a partir de una función de vista.

### La función *url_for()*.

La función *url_for()* permite utilizar una función de vista como referencia en vez de su URL correspondiente con la siguiente sintaxis.

``` python
url_for(<nombre de la función>, values=<argumentos de la función>)
```

Dicha función cuenta además con algunos otros parámetros adicionales para definir métodos y referencias para la URL.

### El uso de la función *url_for()* para archivos estáticos.

Flask está configurado para utiliza el directorio [*static*](static) para acceder a archivos estáticos de tal forma que la función *url_for()* permite generar URL a archivos estáticos con la siguiente sintaxis:

```
url_for('static', filename='<nombre del archivo>')
```

**Ejemplo:**

Se correrá una aplicación con dos funciones de vista.
* La función de vista *index()* apunta http://localhost:5000.
* La función de vista *usuario()* es dinámica para ```http://localhost:5000/<cadena de caracteres>``` despliega código HTML que utiliza a la función *url_for()* para la función *index()* y para un archivo estático.

La URL http://localhost:5000/Juan ejemplifica el uso de la función *usuario()*.

**Advertencia:** Una vez ejecutada la siguiente celda, es necesario interrumpir el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
from flask import url_for, Flask

app = Flask(__name__)

@app.route('/')
def index():
    return '<h1>Este es el inicio.</h1>' 
    
@app.route('/<nombre>')
def usuario(nombre):
    return '<h1>Hola, {}.</h1>\
    <ul>\
    <li> Regresar al <a href={}>inicio.</a></li>\
    <li> Ver la <a href={}>imagen.</a></li>\
    </ul>'.format(nombre, url_for('index'), url_for('static', filename='pythonista.png'))

#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run('0.0.0.0')

## Envío de mensajes.

Los siguientes componentes del paquete *flask* permiten interactuar con el cliente mediante envío de mensajes y redireccionamiento del navegador a URL específicos. 

### La función *flash()*.

Es posible enviar un mensaje al contexto de petición de una sesión activa mediante la función flash con la siguiente sintaxis:

``` python
flash (<objeto str>, category = <objeto str>)
```
Con esta función se envía un mensaje al cliente después de una petición para ser recibido en la siguiente petición. 

De igual manera y de forma opcional es posible enviar un segundo mensaje, indicando la naturaleza del mensaje principal mediante el parámetro *category*. 

Aún cuando se puede enviar cualquier cadena de caracteres para el parámetro *category*, se recomiendan las siguientes:

* *'message'*
* *'warning'*
* *'error'*
* *'info'*

### La función  *get_flashed_messages()*.

La función *get_flashed_messages()* captura los mensajes enviados por *flash()*. 

Dicha función puede ser ejecutada desde una plantilla con laa siguiente sintaxis:

``` python
get_flashed_messages(with_categories=<objeto tipo bool>, category_filter=[lista de objetos str])
```
Si se utiliza la función *get_flashed_messages()* sin argumentos, ésta capturará sólo el mensaje principal en vista de que el parámetro  *with_categories* tiene un valor por defecto de _False_.

El parámetro *category_filter* indicará a la función la categoría de los mensajes que deberá capturar. 

Si no se indica nada, se capturarán todos los mensajes.


### La función *redirect()*.

Dicha función le indica al cliente que se redirija a una URL específica.

``` python
redirect('<URL>')
```

## Ejemplo de uso de *flash()*, *get_flashed_messages()* y *redirect()*.

Se utilizará la aplicación del capítulo previo de captura de datos personales de un alumno y una vez que se ingresen los datos correctamente ocurrirá lo siguiente:
* Se enviará un mensaje mediante la función *flash()* indicando que se ingresaron los datos correctamente.
* Se recargará la URL correspondiente a la función de vista *altas()*.
* Se desplegará el mensaje.

La aplicación se desplegará desde http://localhost:5000/altas

Se creó la plantilla [*templates/captura_alumno.html*](templates/captura_alumno.html) con el siguiente código.

``` html
<h1> Datos personales del alumno </h1>
{% from "_formhelpers.html" import render_field %}
<form method="POST">
    {{ form.hidden_tag() }}
    {{ render_field(form.nombre) }}
    {{ render_field(form.primer_apellido) }}
    {{ render_field(form.segundo_apellido) }}
    {{ render_field(form.carrera)}}
    {{ render_field(form.semestre)}}
    {{ render_field(form.promedio)}}
    {{ render_field(form.alcorriente)}}
    <p>
    {{ form.enviar }}
</form>
{% with mensajes = get_flashed_messages() %}
  {% if mensajes %}
    <ul>
    {% for mensaje in mensajes %}
        <li><b>{{ mensaje }}</b></li>
    {% endfor %}
    </ul>
  {% endif %}
{% endwith %}
```
**Advertencia:** Una vez ejecutada la siguiente celda, es necesario interrumpir el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, BooleanField, SelectField
from wtforms.validators import DataRequired, ValidationError
from flask import Flask, render_template, flash, redirect, url_for, abort

carreras = (('Derecho', 'Derecho'), ('Medicina', 'Medicina'), ('Sistemas', 'Sistemas'), ('Diseño', 'Diseño'))

        
class DatosEstudiante(FlaskForm):

            
    def valida_promedio(form, field):
        try:
            numero = float(field.data)
        except:
            raise ValidationError('Debe de ingresar un número')
        if numero < 0 or numero > 10:
            raise ValidationError('Debe de ingresar un número entre 0 y 10')        
            
            
    nombre = StringField('Nombre', [DataRequired()], default = '')
    primer_apellido = StringField('Primer apellido', [DataRequired()], default = '')
    segundo_apellido = StringField('Segundo apellido', default = '')
    carrera = SelectField('Carrera', [DataRequired()], choices = carreras)
    semestre = SelectField('Semestre', [DataRequired()], choices = [(str(x), str(x)) for x in range(1, 50)])
    promedio = StringField('Promedio', [DataRequired(), valida_promedio], default = '0')
    alcorriente = BooleanField('Al corriente de pagos')
    enviar = SubmitField('Enviar')
    
            
app = Flask(__name__)
app.config['SECRET_KEY']='Saludines'



@app.route('/altas', methods=['GET', 'POST'])
def altas():
    bandera = True
    forma = DatosEstudiante()
    if  forma.validate_on_submit():
        flash('Datos ingresados correctamente')
        for campo in ['nombre', 'primer_apellido', 'segundo_apellido', 'carrera',
                     'semestre', 'promedio', 'alcorriente']:
            print(forma[campo].data) 
        return redirect(url_for('altas'))
    return render_template('captura_alumno.html', form=forma)

#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run('0.0.0.0')

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>